In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string 

In [3]:
fake=pd.read_csv("Fake.csv")
true=pd.read_csv("True.csv")

In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake["class"]=0
true["class"]=1

In [7]:
fake.head() #after adding class 0 for fake

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
true.head() #after labeling class as 1 for true 

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [9]:
data=pd.concat([fake,true],axis=0)
data

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [10]:
data.sample(10) # random 10 samples after concatination 

,title,text,subject,date,class
19568,COMEDIAN TIM ALLEN On Hollywood Hypocrites: Th...,https://youtu.be/0J4xPRYbsLU,left-news,"Nov 22, 2016",0
12785,"Britain, EU agree on divorce bill, Northern Ir...",BRUSSELS (Reuters) - Britain and the European ...,worldnews,"December 8, 2017",1
4548,Ex-Trump adviser Flynn talking to Congress abo...,"((In March 30 item, corrects spelling of Kisl...",politicsNews,"March 31, 2017",1
19440,Pakistan army pushed political role for milita...,"LAHORE, Pakistan (Reuters) - The backing of a ...",worldnews,"September 21, 2017",1
9340,OOPS! DOCUMENTS SHOW Roy Moore Represented Acc...,"Court documents related to Tina Johnson, an Al...",politics,"Nov 27, 2017",0
5519,Trump says Israel should be 'reasonable' in pe...,JERUSALEM (Reuters) - U.S. President Donald Tr...,politicsNews,"February 10, 2017",1
11856,WATCH HILARIOUSLY AWKWARD Moment Between Rick ...,Franken asked Perry if he enjoyed a meeting th...,politics,"Jan 20, 2017",0
4469,Guy Who Forgot 9/11 Happened Says Hillary Is ...,Former New York City Mayor Rudy Giuliani decid...,News,"September 27, 2016",0
19313,WOW! Christian Author Gives UNEXPECTED And BRI...,Brigitte Gabriel was born in the Marjeyoun Dis...,left-news,"Jan 13, 2017",0
12556,LOL! PHOTO Accompanying Google Search Of “Path...,The truth hurts The idea that there are peopl...,politics,"Oct 31, 2016",0


In [11]:
print(data.columns)


Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')


In [12]:
data=data.drop(["title","subject","date"],axis=1)

In [13]:
data.reset_index(inplace=True)

In [14]:
data.drop(["index"],axis=1,inplace=True)

In [15]:
data.sample(5)

,text,class
20578,We all know Bill Clinton is a sexual predator....,0
31638,(Reuters) - New York Attorney General Eric Sch...,1
39145,DUBAI (Reuters) - Saudi Arabian investment fir...,1
1272,"On Monday, Donald Trump once again attacked Lo...",0
8777,We recently saw the heartbreaking plea of 15-y...,0


In [17]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub('\[.*?\]', "", text)  # Remove text in square brackets
    text = re.sub('\\W', " ", text)  # Remove non-word characters
    text = re.sub("https?://\S+|www\.\S+", "", text)  # Remove URLs
    text = re.sub("<.*?>+", "", text)  # Remove HTML tags
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)  # Remove punctuation
    text = re.sub("\n", " ", text)  # Remove newlines
    text = re.sub("\w*\d\w*", "", text)  # Remove words with numbers
    return text

In [18]:
data['text']=data["text"].apply(clean_text)

In [19]:
x=data["text"]
y=data["class"]
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [20]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)


In [21]:
lr =LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [22]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [23]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [24]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']